# Starter Classes

All neccesary classes and start of project

In [63]:
#All neccesary classes for project

#general
import seaborn as sns
import pandas as pd
import numpy as np

#for preprocessing
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import Imputer
from sklearn.pipeline import FeatureUnion
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import roc_curve

#for machine learning
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import expon, reciprocal
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsClassifier
from sklearn.multiclass import OneVsOneClassifier

#for evaluation
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score

#for utility packages
from Utilities.utilities import import_data
from Utilities.utilities import DataFrameSelector
from Utilities.utilities import CategoricalEncoder
from Utilities.utilities import display_scores

In [2]:
#Get Data

raw_data = import_data("cleaned_raw_2.csv")
raw_data.head()

,cell.line,treatment.status,Smad2.S245.250.255,Smad2.S465.467,eIF4E.S209,eIF4G.S1108,eNOS.NOS.III.S116,FAKY576.577,FKHR.FOX01.S256,FKHR.FOX01.T24.FKHRL1.FOX03a.T32,...,LC3B,LDHA,Mn.Superoxide.Dismutase,PI3.Kinase.p110gamma,Proteasome.20S.C2,PTEN.TOTAL,PUMA,ST6GALNAC5,Vimentin,p62.SQSTM1
0,HPAC,1,11047.95,9063.41,23155.79,4568.77,16350.27,13711.64,17292.02,20599.04,...,2887.08,20599.04,25591.09,9837.93,23623.56,35242.23,22026.47,18844.94,1040.02,3862.23
1,HPAC,1,12185.48,5721.58,22026.47,2927.78,15740.61,13752.84,16797.75,20070.36,...,2356.66,20931.29,26108.08,9172.82,21785.51,33523.44,20434.91,16564.22,924.27,3619.17
2,HPAC,1,18996.31,9877.35,26108.08,3219.56,17051.62,10178.16,14661.80,16966.57,...,3622.79,23155.79,27722.50,9779.07,20496.32,29732.62,16139.10,17171.40,882.71,3991.81
3,HPAC,0,16026.52,9320.77,25591.09,9209.59,19712.34,12531.49,17535.81,20292.37,...,3714.50,21611.91,21982.46,10311.35,25336.48,31888.47,14185.85,19110.63,954.32,4661.07
4,HPAC,0,17378.68,8760.43,25336.48,7193.98,17800.82,12926.05,17676.65,19970.28,...,2858.35,21960.48,24587.65,9604.63,21504.12,32208.97,12644.78,16597.38,966.81,3956.04


# Prepare Data for Machine Learning

This work flow will break apart a training set and test set as well as put the data through a pipeline to prepare it for fitting to a model 

In [10]:
split = StratifiedShuffleSplit(n_splits=1, test_size=0.3, random_state=42)
for train_index, test_index in split.split(raw_data, raw_data["cell.line"], raw_data["treatment.status"]):
    strat_train_set = raw_data.loc[train_index]
    strat_test_set = raw_data.loc[test_index]
    
strat_train_set_dmso = strat_train_set.loc[raw_data['treatment.status'] == 0]
strat_train_set_dmso = strat_train_set_dmso.drop('treatment.status', axis=1)

strat_test_set_dmso = strat_test_set.loc[raw_data['treatment.status'] == 0]
strat_test_set_dmso = strat_test_set_dmso.drop('treatment.status', axis=1)
    
strat_train_set_dmso

,cell.line,Smad2.S245.250.255,Smad2.S465.467,eIF4E.S209,eIF4G.S1108,eNOS.NOS.III.S116,FAKY576.577,FKHR.FOX01.S256,FKHR.FOX01.T24.FKHRL1.FOX03a.T32,LKB1.S334,...,LC3B,LDHA,Mn.Superoxide.Dismutase,PI3.Kinase.p110gamma,Proteasome.20S.C2,PTEN.TOTAL,PUMA,ST6GALNAC5,Vimentin,p62.SQSTM1
21,SW1990,11626.01,10614.76,18769.72,31571.17,26635.48,13697.92,23388.52,37797.57,25336.48,...,24100.80,23155.79,20723.01,13399.87,23388.52,18694.80,15048.00,17430.90,1408.10,3673.87
81,HUPT3,32208.97,22697.27,38561.14,21141.64,26108.08,30946.03,34891.55,34891.55,26903.18,...,28282.54,24587.65,23860.98,20994.16,14574.08,9595.02,18324.61,34891.55,1619.71,5619.51
5,HPAC,24587.65,6317.00,24100.80,7800.94,18015.72,12926.05,14898.27,16547.66,20455.35,...,3401.60,18453.33,25591.09,8708.02,16531.13,29143.87,10981.86,16932.67,891.58,3684.90
68,A818,12657.44,4628.55,23623.56,14342.75,28566.79,15552.87,17000.54,24100.80,31257.05,...,3952.09,18601.55,23623.56,13629.62,27446.67,28282.54,13794.15,20110.56,1152.86,6634.25
56,PK8,16780.95,3522.76,15914.73,20070.36,28282.54,12544.03,19653.28,19225.64,21547.17,...,6002.91,25336.48,24587.65,11510.33,26108.08,24100.80,13521.01,17800.82,983.38,3703.38
49,PATC43,30031.45,6809.00,12258.80,54720.84,20578.46,22004.44,24834.77,19910.45,27446.67,...,14764.79,21785.51,19614.02,11452.92,22247.84,18958.36,12977.85,25591.09,NaN,1828.04
33,Pa14C,23155.79,14486.90,26635.48,42616.63,25848.29,13266.54,24343.02,35954.15,15803.71,...,18638.79,22247.84,22471.44,12161.13,NaN,NaN,NaN,NaN,NaN,NaN
27,Pa01C,19148.88,7442.78,18342.95,27446.67,21655.18,13108.29,17553.34,30638.11,19418.85,...,5636.40,23860.98,13670.56,11047.95,NaN,26108.08,9887.24,16106.86,1310.29,2960.16
54,PK8,19283.39,7317.32,27446.67,30031.45,34200.64,25084.37,29436.77,32532.68,28566.79,...,10239.42,33860.36,27446.67,11660.94,30638.11,26903.18,15184.04,18269.71,1301.15,3334.24
55,PK8,15048.00,3035.10,10016.61,16983.54,22697.27,9090.63,11813.52,15032.95,19930.36,...,8160.00,24343.02,29143.87,13359.73,22925.38,22925.38,15490.77,19614.02,999.25,2963.13


In [44]:
#Isolate X and Y Variable from the training set

raw_data_fix = strat_train_set_dmso.drop("cell.line", axis=1)
data_labels = strat_train_set_dmso["cell.line"].copy()


num_attribs = list(raw_data_fix)
cat_attribs = ["cell.line"]

raw_data_fix

,Smad2.S245.250.255,Smad2.S465.467,eIF4E.S209,eIF4G.S1108,eNOS.NOS.III.S116,FAKY576.577,FKHR.FOX01.S256,FKHR.FOX01.T24.FKHRL1.FOX03a.T32,LKB1.S334,mTOR.S2448,...,LC3B,LDHA,Mn.Superoxide.Dismutase,PI3.Kinase.p110gamma,Proteasome.20S.C2,PTEN.TOTAL,PUMA,ST6GALNAC5,Vimentin,p62.SQSTM1
21,11626.01,10614.76,18769.72,31571.17,26635.48,13697.92,23388.52,37797.57,25336.48,38561.14,...,24100.80,23155.79,20723.01,13399.87,23388.52,18694.80,15048.00,17430.90,1408.10,3673.87
81,32208.97,22697.27,38561.14,21141.64,26108.08,30946.03,34891.55,34891.55,26903.18,32532.68,...,28282.54,24587.65,23860.98,20994.16,14574.08,9595.02,18324.61,34891.55,1619.71,5619.51
5,24587.65,6317.00,24100.80,7800.94,18015.72,12926.05,14898.27,16547.66,20455.35,28853.90,...,3401.60,18453.33,25591.09,8708.02,16531.13,29143.87,10981.86,16932.67,891.58,3684.90
68,12657.44,4628.55,23623.56,14342.75,28566.79,15552.87,17000.54,24100.80,31257.05,31571.17,...,3952.09,18601.55,23623.56,13629.62,27446.67,28282.54,13794.15,20110.56,1152.86,6634.25
56,16780.95,3522.76,15914.73,20070.36,28282.54,12544.03,19653.28,19225.64,21547.17,33523.44,...,6002.91,25336.48,24587.65,11510.33,26108.08,24100.80,13521.01,17800.82,983.38,3703.38
49,30031.45,6809.00,12258.80,54720.84,20578.46,22004.44,24834.77,19910.45,27446.67,33189.87,...,14764.79,21785.51,19614.02,11452.92,22247.84,18958.36,12977.85,25591.09,NaN,1828.04
33,23155.79,14486.90,26635.48,42616.63,25848.29,13266.54,24343.02,35954.15,15803.71,36315.50,...,18638.79,22247.84,22471.44,12161.13,NaN,NaN,NaN,NaN,NaN,NaN
27,19148.88,7442.78,18342.95,27446.67,21655.18,13108.29,17553.34,30638.11,19418.85,33523.44,...,5636.40,23860.98,13670.56,11047.95,NaN,26108.08,9887.24,16106.86,1310.29,2960.16
54,19283.39,7317.32,27446.67,30031.45,34200.64,25084.37,29436.77,32532.68,28566.79,33860.36,...,10239.42,33860.36,27446.67,11660.94,30638.11,26903.18,15184.04,18269.71,1301.15,3334.24
55,15048.00,3035.10,10016.61,16983.54,22697.27,9090.63,11813.52,15032.95,19930.36,25591.09,...,8160.00,24343.02,29143.87,13359.73,22925.38,22925.38,15490.77,19614.02,999.25,2963.13


In [12]:
num_pipeline = Pipeline([
    ('selector', DataFrameSelector(num_attribs)),
    ('imputer', Imputer(strategy="median")),
    ('std_scaler', StandardScaler()),
])

cat_pipeline = Pipeline([
    ('selector', DataFrameSelector(cat_attribs)),
    ('cat_encoder', CategoricalEncoder(encoding="onehot-dense")),
])

full_pipeline = FeatureUnion(transformer_list=[
    ("num_pipeline", num_pipeline),
    ("cat_pipeline", cat_pipeline),
])

In [13]:
#Run the data through pipeline so now it is ready to use in ML algorithms

prepared_data = num_pipeline.fit_transform(raw_data_fix)
pd.DataFrame(prepared_data).head()

,0,1,2,3,4,5,6,7,8,9,...,147,148,149,150,151,152,153,154,155,156
0,-1.045516,-0.013989,-0.374870,0.286446,0.430379,-0.335180,0.360000,1.780523,0.324160,0.430913,...,1.454286,-0.268320,-0.605881,0.195208,0.185324,-0.616836,0.251905,-0.770439,-0.163423,-0.104291
1,1.619711,0.656776,2.307075,-0.473635,0.351409,2.809652,2.139919,1.380133,0.597291,-0.106193,...,2.003008,-0.000738,-0.107059,3.095237,-1.949153,-2.049859,1.294621,3.586436,-0.035603,1.044571
2,0.632848,-0.252581,0.347547,-1.445876,-0.860300,-0.475915,-0.953737,-1.147277,-0.526791,-0.433954,...,-1.261834,-1.147104,0.167965,-1.596467,-1.475240,1.028670,-1.042063,-0.894760,-0.475421,-0.097778
3,-0.911960,-0.346315,0.282876,-0.969123,0.719564,0.003031,-0.628443,-0.106610,1.356323,-0.191859,...,-1.189600,-1.119405,-0.144800,0.282943,1.168032,0.893029,-0.147108,-0.101796,-0.317598,1.643755
4,-0.378018,-0.407704,-0.761752,-0.551708,0.677002,-0.545568,-0.217972,-0.778306,-0.336449,-0.017921,...,-0.920494,0.139201,0.008455,-0.526350,0.843883,0.234494,-0.234029,-0.678135,-0.419970,-0.086866


# Ready for Machine Learning:

At this point the data is in the proper format for training your model. Relevant variables listed below:

kirsten_data_prepared - The X in your model. These are what we are going to use to make predictions. kirsten_label - The Y in your model. These are the values you are going to try and predict.

strat_train_set - Contains the full data set for training. Can be used to extract a variable. Rerun with full_pipeline.tranform_fit to prepare your X for model

strat_test_set - Contains the full data set for testing model. See below for how to use it verification

# Random Forest Model with GridSearchCV and RandomSearchCV


Uses a random forest and GridSearchCV to predict the hyperparameters and generate a model

In [14]:
#fit a basic forest regression to ensure data is in proper format

ovo_clf = OneVsOneClassifier(RandomForestClassifier(random_state=42))
ovo_clf.fit(prepared_data, data_labels)
predictions = ovo_clf.predict(prepared_data)

confusion_matrix(data_labels, predictions)

array([[2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2]], dtype=int64)

In [16]:
f1_score(data_labels, predictions, average="macro")

c:\users\sdgeo\dropbox\derlab~1\data\kirste~1\env\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


0.9145299145299146

In [17]:
data_labels

21     SW1990
81      HUPT3
5        HPAC
68       A818
56        PK8
49     PATC43
33      Pa14C
27      Pa01C
54        PK8
55        PK8
10      PANC1
74        PSN
61    TCCPAN2
16     HPAFII
9       PANC1
15     HPAFII
73        PSN
20     SW1990
3        HPAC
38      Pa04C
67       A818
40      Pa04C
50     PATC43
46     PATC43
79      HUPT3
48     PATC43
62    TCCPAN2
34      Pa14C
57        PK8
75        PSN
Name: cell.line, dtype: object

In [33]:
#Do optomization by doing a grid search

param_grid = [
    {'n_estimators': [80,100,120], 'max_features': [50,60,70,80,90,100]},
    {'bootstrap': [False], 'n_estimators': [80,100,120], 'max_features': [50,60,70,80,90,100]}
]

forest_clf = RandomForestClassifier()

for_search = GridSearchCV(forest_clf, param_grid, cv=3, scoring='accuracy')
for_search.fit(prepared_data, data_labels)

GridSearchCV(cv=3, error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid=[{'n_estimators': [80, 100, 120], 'max_features': [50, 60, 70, 80, 90, 100]}, {'bootstrap': [False], 'n_estimators': [80, 100, 120], 'max_features': [50, 60, 70, 80, 90, 100]}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='accuracy', verbose=0)

In [39]:
items = for_search.predict(prepared_data)
f1_score(data_labels, items, average="macro")

1.0

In [18]:
#Do a random search for best iterations

param_grid_random = {
        'bootstrap': [True, False],
        'n_estimators': list(range(50,150)),
        'max_features': list(range(1,150)),
    }

forest_clf_rand = RandomForestClassifier()

for_random = RandomizedSearchCV(forest_clf_rand, param_distributions=param_grid_random,
                                n_iter=3500, cv=3, scoring='accuracy',
                                verbose=2, n_jobs=8, random_state=42)

for_random.fit(prepared_data, data_labels)

Fitting 3 folds for each of 3500 candidates, totalling 10500 fits


c:\users\sdgeo\dropbox\derlab~1\data\kirste~1\env\lib\site-packages\sklearn\model_selection\_split.py:605: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=3.
  % (min_groups, self.n_splits)), Warning)
[Parallel(n_jobs=8)]: Done  25 tasks      | elapsed:    4.8s
[Parallel(n_jobs=8)]: Done 146 tasks      | elapsed:    8.8s
[Parallel(n_jobs=8)]: Done 349 tasks      | elapsed:   15.3s
[Parallel(n_jobs=8)]: Done 632 tasks      | elapsed:   25.4s
[Parallel(n_jobs=8)]: Done 997 tasks      | elapsed:   37.7s
[Parallel(n_jobs=8)]: Done 1442 tasks      | elapsed:   53.6s
[Parallel(n_jobs=8)]: Done 1969 tasks      | elapsed:  1.2min
[Parallel(n_jobs=8)]: Done 2576 tasks      | elapsed:  1.6min
[Parallel(n_jobs=8)]: Done 3265 tasks      | elapsed:  2.0min
[Parallel(n_jobs=8)]: Done 4034 tasks      | elapsed:  2.5min
[Parallel(n_jobs=8)]: Done 4885 tasks      | elapsed:  3.1min
[Parallel(n_jobs=8)]

RandomizedSearchCV(cv=3, error_score='raise',
          estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
          fit_params=None, iid=True, n_iter=3500, n_jobs=8,
          param_distributions={'bootstrap': [True, False], 'n_estimators': [50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106,...130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149]},
          pre_dispatch='2*n_jobs', ra

# Evaluate Fits 

Test to see if the fits are good and give metrics

In [36]:
search_found = for_random

In [50]:
search_found.best_params_

{'bootstrap': True, 'max_features': 36, 'n_estimators': 128}

In [21]:
cvres = search_found.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(np.sqrt(-mean_score), params)

c:\users\sdgeo\dropbox\derlab~1\data\kirste~1\env\lib\site-packages\ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in sqrt
  This is separate from the ipykernel package so we can avoid doing imports until


nan {'n_estimators': 89, 'max_features': 14, 'bootstrap': False}
nan {'n_estimators': 69, 'max_features': 120, 'bootstrap': True}
nan {'n_estimators': 71, 'max_features': 144, 'bootstrap': True}
nan {'n_estimators': 101, 'max_features': 149, 'bootstrap': False}
nan {'n_estimators': 90, 'max_features': 22, 'bootstrap': True}
nan {'n_estimators': 62, 'max_features': 59, 'bootstrap': False}
nan {'n_estimators': 127, 'max_features': 18, 'bootstrap': False}
nan {'n_estimators': 90, 'max_features': 87, 'bootstrap': False}
nan {'n_estimators': 147, 'max_features': 113, 'bootstrap': True}
nan {'n_estimators': 106, 'max_features': 138, 'bootstrap': False}
nan {'n_estimators': 145, 'max_features': 129, 'bootstrap': True}
nan {'n_estimators': 123, 'max_features': 35, 'bootstrap': True}
nan {'n_estimators': 129, 'max_features': 67, 'bootstrap': False}
nan {'n_estimators': 109, 'max_features': 59, 'bootstrap': False}
nan {'n_estimators': 92, 'max_features': 65, 'bootstrap': True}
nan {'n_estimators

nan {'n_estimators': 108, 'max_features': 37, 'bootstrap': True}
nan {'n_estimators': 52, 'max_features': 122, 'bootstrap': True}
nan {'n_estimators': 134, 'max_features': 50, 'bootstrap': True}
nan {'n_estimators': 82, 'max_features': 140, 'bootstrap': False}
nan {'n_estimators': 129, 'max_features': 96, 'bootstrap': True}
nan {'n_estimators': 89, 'max_features': 92, 'bootstrap': False}
nan {'n_estimators': 85, 'max_features': 13, 'bootstrap': False}
nan {'n_estimators': 110, 'max_features': 73, 'bootstrap': False}
nan {'n_estimators': 138, 'max_features': 16, 'bootstrap': True}
nan {'n_estimators': 52, 'max_features': 144, 'bootstrap': True}
nan {'n_estimators': 148, 'max_features': 130, 'bootstrap': False}
nan {'n_estimators': 77, 'max_features': 126, 'bootstrap': True}
nan {'n_estimators': 73, 'max_features': 20, 'bootstrap': True}
nan {'n_estimators': 65, 'max_features': 137, 'bootstrap': False}
nan {'n_estimators': 84, 'max_features': 136, 'bootstrap': True}
nan {'n_estimators': 

nan {'n_estimators': 76, 'max_features': 85, 'bootstrap': True}
nan {'n_estimators': 99, 'max_features': 60, 'bootstrap': False}
nan {'n_estimators': 105, 'max_features': 30, 'bootstrap': True}
nan {'n_estimators': 118, 'max_features': 96, 'bootstrap': True}
nan {'n_estimators': 79, 'max_features': 139, 'bootstrap': False}
nan {'n_estimators': 146, 'max_features': 39, 'bootstrap': False}
nan {'n_estimators': 132, 'max_features': 53, 'bootstrap': True}
nan {'n_estimators': 149, 'max_features': 123, 'bootstrap': True}
nan {'n_estimators': 102, 'max_features': 37, 'bootstrap': False}
nan {'n_estimators': 80, 'max_features': 3, 'bootstrap': True}
nan {'n_estimators': 122, 'max_features': 56, 'bootstrap': True}
nan {'n_estimators': 108, 'max_features': 99, 'bootstrap': False}
nan {'n_estimators': 142, 'max_features': 102, 'bootstrap': True}
nan {'n_estimators': 128, 'max_features': 36, 'bootstrap': False}
nan {'n_estimators': 55, 'max_features': 125, 'bootstrap': True}
nan {'n_estimators': 

In [22]:
attributes = num_attribs

feature_importances = search_found.best_estimator_.feature_importances_
sorted(zip(feature_importances, attributes), reverse=True)

[(0.025245998341675614, 'X4EBP1.T70'),
 (0.02201866907958871, 'PARP.cleaved.D214'),
 (0.021831143638042783, 'X4EBP1.S65'),
 (0.02151457665640201, 'E.Cadherin'),
 (0.020098612949846724, 'Ras.GRF1.S916'),
 (0.019438650422906434, 'p53.S15'),
 (0.017161496286864312, 'c.Myc'),
 (0.014076845077140509, 'AR.S650'),
 (0.013934867235233078, 'GSK.3alpha.Y279.beta.Y216'),
 (0.013377884870728746, 'p70S6.Kinase.T389'),
 (0.013340382316367024, 'PDK1.S241'),
 (0.013313008810093677, 'eIF2alpha.S51'),
 (0.012482667061497834, 'Smad2.S245.250.255'),
 (0.012439647447813616, 'GSK.3a.B.S21.9'),
 (0.011904235694158638, 'Caspase.7.Cleaved.D198'),
 (0.011829492969566395, 'Catenin.beta.T41.S45'),
 (0.011738187743867725, 'Cofilin.S3'),
 (0.011616446009484666, 'MARCKS.S152.156'),
 (0.011444143847348681, 'PLC.gamma.1'),
 (0.011442633954039808, 'Bcl.2'),
 (0.010463129686970272, 'BLVRB..Biliverdin.Reductase.B.'),
 (0.010249531004546831, 'p38.MAPK.T180.182'),
 (0.010066196595432978, 'Bcl.xL'),
 (0.009990285032562288, 

# KNN nearest neighbors model

Here are going to build yet another model and find how good its fit is

In [23]:
#test a basic model using cross_val_score

knn_clf = KNeighborsClassifier()

knn_clf.fit(prepared_data, data_labels)
cell_scores = cross_val_score(knn_clf, prepared_data, data_labels, scoring="accuracy", cv=3)
knn_clf

c:\users\sdgeo\dropbox\derlab~1\data\kirste~1\env\lib\site-packages\sklearn\model_selection\_split.py:605: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=3.
  % (min_groups, self.n_splits)), Warning)


KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform')

In [62]:
#Use random search here to optomize SVM

param_grid_random = {
        'algorithm': ['ball_tree', 'kd_tree', 'brute'],
        'n_jobs': [8],
        'leaf_size': list(range(20,40)),
        'weights': ['uniform', 'distance'],
    }

knn_clf = KNeighborsClassifier()
knn_search = RandomizedSearchCV(knn_clf, param_distributions=param_grid_random,
                                n_iter=120, cv=3, scoring='accuracy',
                                verbose=2, n_jobs=8, random_state=42)
knn_search.fit(prepared_data, data_labels)

Fitting 3 folds for each of 120 candidates, totalling 360 fits


c:\users\sdgeo\dropbox\derlab~1\data\kirste~1\env\lib\site-packages\sklearn\model_selection\_split.py:605: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=3.
  % (min_groups, self.n_splits)), Warning)
[Parallel(n_jobs=8)]: Done  25 tasks      | elapsed:    4.9s
[Parallel(n_jobs=8)]: Done 146 tasks      | elapsed:    8.1s
[Parallel(n_jobs=8)]: Done 360 out of 360 | elapsed:   10.8s finished


RandomizedSearchCV(cv=3, error_score='raise',
          estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform'),
          fit_params=None, iid=True, n_iter=120, n_jobs=8,
          param_distributions={'algorithm': ['ball_tree', 'kd_tree', 'brute'], 'n_jobs': [8], 'leaf_size': [20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39], 'weights': ['uniform', 'distance']},
          pre_dispatch='2*n_jobs', random_state=42, refit=True,
          return_train_score='warn', scoring='accuracy', verbose=2)

In [67]:
items = knn_search.predict(prepared_data)
f1_score(data_labels, items, average="macro")

1.0

In [68]:
knn_search.best_params_

{'algorithm': 'ball_tree', 'leaf_size': 20, 'n_jobs': 8, 'weights': 'distance'}

# Test the parameters you would like to see how well any regression performs

Here you can test your differnt models to see how well the model fits the test data 

In [2]:
knn_final = knn_search.best_estimator_

NameError: name 'knn_search' is not defined

In [1]:
#prepare and select data from test set

#final_model = search_found
final_model = knn_final

X_test = strat_test_set.drop("cell.line", axis=1)
Y_test = strat_test_set["cell.line"].copy()

X_test = X_test.drop("treatment.status", axis=1)

X_test_prepared = num_pipeline.fit_transform(X_test)

NameError: name 'knn_final' is not defined

In [58]:
#Look at predictions rmse

final_predictions = final_model.predict(X_test_prepared)


In [59]:
confusion_matrix(Y_test, final_predictions)

array([[2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1]], dtype=int64)

In [60]:
score = f1_score(Y_test, final_predictions, average="macro")
print(score)

0.958974358974359
